In [16]:
import requests
import json
from azure.storage.blob import BlobServiceClient,generate_blob_sas,BlobSasPermissions
from dateutil.relativedelta import relativedelta

In [17]:
def get_sas_link(client, container_name, blob_name):

    image =  client.get_blob_client(container=container_name, blob=blob_name)

    token = generate_blob_sas(account_name=user,
                                  account_key = key,
                                  container_name = container_name,
                                  blob_name= blob_name.replace('\\','/'),
                                  permission=BlobSasPermissions(read=True, tag=False),
                                  expiry=datetime.utcnow() + relativedelta(months=3)
                                  )

    return image.url + "?" + token

def get_result_from_api(url_A, url_B):
    
    url = "http://127.0.0.1:8000/check_duplciate_image"

    headers = {
        "Content-Type": "application/json"
    }

    data = {
        "url_image_A" : url_A,
        "url_image_B" : url_B
    }

    response = requests.get(url, headers=headers, data=json.dumps(data))
    
    return response



In [ ]:
f = open('config.json','r')
config = json.load(f)
user = config['username']
key  = config['password']

blob_client = BlobServiceClient(account_url=f"https://{user}.blob.core.windows.net",
                                credential=key)

url_A = get_sas_link(blob_client,container_name = 'container_name',
                   blob_name= 'file_path_A')
url_B = get_sas_link(blob_client,container_name = 'container_name',
                   blob_name= 'file_path_B')

result = get_result_from_api(url_A, url_B)
result.json()